# Importings

- consider imputing some of the tests
- take the LGBM or Catboost Performance o
- try different values for clipping

In [65]:
import pandas as pd                                    
import numpy as np                                     
from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error         
from sklearn.model_selection import GroupKFold,KFold, TimeSeriesSplit   # for validation
from sklearn.preprocessing import LabelEncoder         # for encoding
import sklearn.manifold._t_sne as tsne                 # for t_sne
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit# for plotting
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from matplotlib import rcParams
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.stats import rankdata
from math import pi
#import xgboost as xgb
#from catboost import CatBoostRegressor
from sklearn.cluster import KMeans
#from path import Path

class CFG:
  data_folder ="/Users/ahmed/Downloads/computerscience/AI-Compeitions/geoai-ground-level-no2-estimation/geoai-ground/data/"

def reading_data(path: str) -> pd.DataFrame:
  train = pd.read_csv(path + 'Train.csv')
  test = pd.read_csv(path + 'Test.csv')
  return train,test

train,test = reading_data(CFG.data_folder)

groups = train['ID']
test_id = test['ID_Zindi']
train_date = train['Date']
test_date = test['Date']
test_id = test['ID_Zindi']
pd.options.display.max_columns = 200
#train = train.dropna(axis=0)
#test = test.dropna(axis=0)

seed = 7 
seed2 = 77
tiney_fraction = 1e-05


In [66]:
train['NO2_total'].max()

0.002047452

In [67]:
#clipping target and outliers 
train = train[train['Precipitation'] < 112.000]
train = train[train['GT_NO2'] < 122]
train = train[train['NO2_total'] <= 0.001947452]
train = train[train['NO2_trop'] <= 0.000905 ]

In [68]:
train['ID'].sort_values()

44161     PD01
84846     PD01
49217     PD01
69757     PD01
72680     PD01
         ...  
77498    X9999
14061    X9999
71336    X9999
32863    X9999
3159     X9999
Name: ID, Length: 48596, dtype: object

In [69]:
train = train.dropna(subset=['GT_NO2'])
train.isnull().sum()
train = train[train['GT_NO2'] < 125]

# Feature Engineering

In [70]:
groups = train['ID']
num_feats = train.select_dtypes(include=['float'])
kmeans = KMeans(n_clusters=2)

In [71]:
lat_min, lat_max = 44.92469405, 45.88973369
lon_min, lon_max = 8.736496578, 12.59068235

num_clusters_lat = 3
num_clusters_lon = 4
lat_step = (lat_max - lat_min) / num_clusters_lat
lon_step = (lon_max - lon_min) / num_clusters_lon
def assign_clusters(row, lat_step, lon_step, lat_min, lon_min):
    lat_cluster = int((row['LAT'] - lat_min) / lat_step)
    lon_cluster = int((row['LON'] - lon_min) / lon_step)
    return lat_cluster, lon_cluster
for dataset in (train, test):
    dataset[['lat_cluster', 'lon_cluster']] = dataset.apply(
        assign_clusters, axis=1, result_type='expand',
        lat_step=lat_step, lon_step=lon_step, lat_min=lat_min, lon_min=lon_min)

In [72]:
#train = train.sort_values('Date').reset_index(drop=True)
#test = test.sort_values('Date').reset_index(drop=True)

def time_related_features(train:pd.DataFrame,test:pd.DataFrame, feature='time') -> pd.DataFrame:
  for df in (train, test):
      df['Date'] = pd.to_datetime(df['Date'])
      df['Day'] = df['Date'].dt.day
      df['Month'] = df['Date'].dt.month
      df['Month_Day'] = df['Month'].astype(str) + '-' + df['Day'].astype(str)
      df['Year'] = df['Date'].dt.year
      df['Weekday'] = df['Date'].dt.weekday
      df['Year_Week'] = df['Year'].astype(str) + '-' + df['Weekday'].astype(str)
      df.drop(columns=['Weekday', 'Year', 'Day', 'ID_Zindi'], inplace=True)
      df['Month_lag1'] = df['Month'].shift(1)
      df['Month_lag2'] = df['Month'].shift(2)
      return train,test

time_related_features(train,test,feature='Date')

/var/folders/4c/h16036bx2ync7wqkqvrl2p1h0000gn/T/ipykernel_18284/1550074779.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])


(            Date      ID        LAT        LON  Precipitation     LST  \
 11    2019-01-01    VI03  45.713559  11.367657            0.0  275.18   
 15    2019-01-01  X10019  45.690429   9.484261            0.0  281.48   
 16    2019-01-01  X10038  45.619244   8.756970            0.0  282.98   
 20    2019-01-01  X10279  45.478996   9.235491            0.0     NaN   
 21    2019-01-01  X10326  45.281936   8.754335            0.0     NaN   
 ...          ...     ...        ...        ...            ...     ...   
 86569 2021-12-31   X6781  45.539529  10.231786            0.0  284.40   
 86571 2021-12-31   X6826  45.463753  10.480772            0.0  283.54   
 86572 2021-12-31   X6859  45.515708  10.336240            0.0  282.10   
 86578 2021-12-31   X9877  45.580263   9.273573            0.0  283.64   
 86581 2021-12-31   X9969  45.842207   9.351658            0.0  284.98   
 
             AAI  CloudFraction  NO2_strat  NO2_total  NO2_trop  \
 11    -0.159398       0.000000   0.000024 

In [73]:
def rolling(feature):
    for dataset in (train,test):
        dataset[f"{feature}_rolling_max_60"] = dataset[feature].rolling(60).max()

rolling('NO2_trop')
rolling('NO2_total')
rolling('TropopausePressure')
rolling('CloudFraction')
rolling('Precipitation')

# Missing Values & Encoding

In [74]:
for df in(train,test):
    df.drop(columns=['Date','ID','Precipitation','CloudFraction','AAI','lat_cluster'], axis=1,inplace=True)

def encoder(train,test):
  le = LabelEncoder()
  for df in(train,test):
      for col in df.columns:
          if df[col].dtype == 'object':
              df[col] = le.fit_transform(df[col])
  return train,test

encoder(train,test)

(             LAT        LON     LST  NO2_strat  NO2_total  NO2_trop  \
 11     45.713559  11.367657  275.18   0.000024   0.000101  0.000084   
 15     45.690429   9.484261  281.48   0.000023   0.000245  0.000288   
 16     45.619244   8.756970  282.98   0.000023   0.000156  0.000133   
 20     45.478996   9.235491     NaN   0.000023   0.000231  0.000177   
 21     45.281936   8.754335     NaN   0.000023   0.000144  0.000115   
 ...          ...        ...     ...        ...        ...       ...   
 86569  45.539529  10.231786  284.40   0.000031   0.000176  0.000145   
 86571  45.463753  10.480772  283.54   0.000031   0.000241  0.000210   
 86572  45.515708  10.336240  282.10   0.000031   0.000212  0.000180   
 86578  45.580263   9.273573  283.64   0.000031   0.000259  0.000228   
 86581  45.842207   9.351658  284.98   0.000031   0.000153  0.000122   
 
        TropopausePressure  GT_NO2  lon_cluster  Month  Month_Day  Year_Week  \
 11            14422.64906  39.000            2      1

# CV and Modeling

In [75]:
n_splits = 5 
gkf = GroupKFold(n_splits=n_splits)
model = LGBMRegressor(random_state=seed)
def validate(trainset,testset,target_col):
    model.fit(trainset.drop(columns=[target_col]),trainset[target_col])
    y_predicted = model.predict(testset.drop(columns=target_col))
    valid_idx = testset[target_col].notna()
    valid_testset = testset[target_col][valid_idx]
    valid_pred = y_predicted[valid_idx]
    print(f"std : {valid_testset}")
    score = mean_squared_error(valid_pred,valid_testset, squared=False)
    print(f"score : {score}")
    
    return score

stds = []
rmse = []
    
for train_index,test_index in gkf.split(train.drop(columns='GT_NO2'), train['GT_NO2'],groups=groups):
    for i in range(1):
        print(f"train shape : {train_index.shape}, test shape:{test_index.shape}")
    train_v, test_v = train.iloc[train_index], train.iloc[test_index]
    stds.append(test_v['GT_NO2'].std())
    rmse.append(validate(trainset=train_v,testset=test_v,target_col='GT_NO2'))

print('RMSE:', np.array(rmse).mean())
print('RMSE std:', np.array(rmse).std())
print('Standard Deviations of the target in validation Fold:', stds)
print('RMSEs Deviations:', rmse)

train shape : (38645,), test shape:(9951,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000667 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2977
[LightGBM] [Info] Number of data points in the train set: 38645, number of used features: 18
[LightGBM] [Info] Start training from score 23.055794
std : 11       39.000
20       44.550
27       30.425
38       25.775
44       28.325
          ...  
86543    44.600
86549    21.300
86553    71.725
86555    18.600
86558    41.950
Name: GT_NO2, Length: 9951, dtype: float64
score : 9.097423122591954
train shape : (38966,), test shape:(9630,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001295 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2

/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std : 15       15.925
16       27.775
35       24.450
41       54.025
49       35.675
          ...  
86540    21.075
86546    46.600
86554    75.500
86572    36.325
86581    28.325
Name: GT_NO2, Length: 9630, dtype: float64
score : 8.422919998584966
train shape : (38641,), test shape:(9955,)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002765 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2977
[LightGBM] [Info] Number of data points in the train set: 38641, number of used features: 18
[LightGBM] [Info] Start training from score 22.662125


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std : 21       30.925
32       61.325
51       40.875
73       50.725
89       32.000
          ...  
86537    50.600
86556    40.125
86569    53.075
86571    34.500
86578    56.275
Name: GT_NO2, Length: 9955, dtype: float64
score : 9.461676326377445
train shape : (39154,), test shape:(9442,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001079 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2976
[LightGBM] [Info] Number of data points in the train set: 39154, number of used features: 18
[LightGBM] [Info] Start training from score 22.561274


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std : 33       59.475
34       37.100
45       35.725
46       27.025
52       20.625
          ...  
86541    58.800
86545    67.475
86550    49.850
86551    44.750
86557    48.200
Name: GT_NO2, Length: 9442, dtype: float64
score : 8.98089549784096
train shape : (38978,), test shape:(9618,)
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000936 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2977
[LightGBM] [Info] Number of data points in the train set: 38978, number of used features: 18
[LightGBM] [Info] Start training from score 22.793069


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std : 25       13.875
31       40.400
37       62.175
43       30.500
47       29.500
          ...  
86535    75.175
86536    39.100
86542    58.725
86544    43.800
86552    52.625
Name: GT_NO2, Length: 9618, dtype: float64
score : 9.155951443680605
RMSE: 9.023773277815186
RMSE std: 0.3398580922339602
Standard Deviations of the target in validation Fold: [15.233196695754797, 15.555797237315929, 18.103354542442755, 18.460153707816172, 16.239050334520144]
RMSEs Deviations: [9.097423122591954, 8.422919998584966, 9.461676326377445, 8.98089549784096, 9.155951443680605]


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [60]:
"""model = CatBoostRegressor(
    iterations=1000,        
    learning_rate=0.1,      
    depth=6,                
    loss_function='RMSE',   
    eval_metric='RMSE',     
    random_seed=seed,          
    verbose=100,
   
)
"""

#model = CatBoostRegressor(random_state=seed)
#model = XGBRegressor(random_state= 7)
model =  LGBMRegressor(random_state=7)
n_splits = 5
n = train['GT_NO2'].count()
num_bins = int(1 + np.log2(n))
cv = GroupKFold(n_splits=n_splits)

def validate(trainset, testset, target_col):

    model.fit(trainset.drop(columns=target_col), trainset[target_col])
    pred = model.predict(testset.drop(columns=target_col))
    valid_idx = testset[target_col].notna()
    valid_testset = testset[target_col][valid_idx]
    valid_pred = pred[valid_idx]
    print('std:', valid_testset.std())
    score = mean_squared_error(valid_testset, valid_pred, squared=False)
    print('score:', score)

    return score
stds = []
rmse = []

for train_idx, test_idx in cv.split(train.drop(columns='GT_NO2'), train['GT_NO2'], groups=groups):
    train_v, test_v = train.iloc[train_idx], train.iloc[test_idx]
    stds.append(test_v['GT_NO2'].std())
    rmse.append(validate(train_v, test_v, 'GT_NO2'))

print('RMSE:', np.array(rmse).mean())
print('RMSE std:', np.array(rmse).std())
print('Standard Deviations:', stds)
print('RMSEs Deviations:', rmse)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000778 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3011
[LightGBM] [Info] Number of data points in the train set: 38645, number of used features: 18
[LightGBM] [Info] Start training from score 22.887798
std: 16.769195455025695
score: 7.274351056827364
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000877 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3013
[LightGBM] [Info] Number of data points in the train set: 38966, number of used features: 18
[LightGBM] [Info] Start training from score 22.916388


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 17.192365223077385
score: 7.281088054392024
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000830 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3012
[LightGBM] [Info] Number of data points in the train set: 38641, number of used features: 18
[LightGBM] [Info] Start training from score 22.906003


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 16.675995828269066
score: 7.252598881837379
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3011
[LightGBM] [Info] Number of data points in the train set: 39154, number of used features: 18
[LightGBM] [Info] Start training from score 22.967780


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 16.41903700061108
score: 7.170948386497015
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000739 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3011
[LightGBM] [Info] Number of data points in the train set: 38978, number of used features: 18
[LightGBM] [Info] Start training from score 22.868936


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


std: 17.00546943616723
score: 7.189021525138602
RMSE: 7.233601580938476
RMSE std: 0.04514241140337197
Standard Deviations: [16.769195455025695, 17.192365223077385, 16.675995828269066, 16.41903700061108, 17.00546943616723]
RMSEs Deviations: [7.274351056827364, 7.281088054392024, 7.252598881837379, 7.170948386497015, 7.189021525138602]


/Users/ahmed/Downloads/computerscience/AirQo/PM2.5-Prediction/AirQo-Virtual-Enviroment/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [61]:
def generate_specifications(model_name, num_folds, model_params, validation_type='StratifiedGroupKFold', shuffle=True):
    specs = []
    specs.append(f"Model: {model_name}")
    specs.append(f"Validation Type: {validation_type}")
    specs.append(f"Number of Folds: {num_folds}")
    specs.append(f"Shuffle: {'Yes' if shuffle else 'No'}")
    
    # Adding model parameters
    specs.append("Model Parameters:")
    for param, value in model_params.items():
        specs.append(f"  {param}: {value}")
    
    # Adding any additional info (if needed)
    specs.append("\nAdditional Notes:")
    specs.append("  -  as the evaluation metric.")
    
    # Joining all specifications into a single string
    return "\n".join(specs)

# Example usage

specifications = generate_specifications(model_name=str(model)[1:10], num_folds=n_splits, model_params=model.get_params())
print(specifications)

Model: GBMRegres
Validation Type: StratifiedGroupKFold
Number of Folds: 5
Shuffle: Yes
Model Parameters:
  boosting_type: gbdt
  class_weight: None
  colsample_bytree: 1.0
  importance_type: split
  learning_rate: 0.1
  max_depth: -1
  min_child_samples: 20
  min_child_weight: 0.001
  min_split_gain: 0.0
  n_estimators: 100
  n_jobs: None
  num_leaves: 31
  objective: None
  random_state: 7
  reg_alpha: 0.0
  reg_lambda: 0.0
  subsample: 1.0
  subsample_for_bin: 200000
  subsample_freq: 0

Additional Notes:
  -  as the evaluation metric.


# inference

In [63]:

model.fit(train.drop(columns='GT_NO2'),train['GT_NO2'])
y_pred = model.predict(test)
#sub_df = pd.DataFrame({'id': test_id,'GT_NO2':y_pred})
#sub_df.to_csv('submission9277409514290884.csv', index=False)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3012
[LightGBM] [Info] Number of data points in the train set: 48596, number of used features: 18
[LightGBM] [Info] Start training from score 22.909476


ValueError: Number of features of the model must match the input. Model n_features_ is 18 and input n_features is 14

In [ ]:
plt.figure(figsize=(16, 7))
sns.lineplot(x=train_date, y=train.GT_NO2, color='blue', label='Train')
sns.lineplot(x=test_date, y=y_pred, color='red', label='Prediction')
plt.legend()
plt.show()

In [ ]:
importances = model.feature_importances_
names = model.feature_names_
fi = pd.DataFrame({'Feature': names,
                   'importances': importances})
fi = fi.sort_values(by='importances', ascending=False)

fi.plot(kind='bar', x='Feature', y='importances', legend=False, figsize=(10, 6))
plt.title('Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

In [ ]:
train.columns